In [ ]:
!pip install pandas==1.3.5
!pip install datasets
!pip install torch
!pip install transformers

     |████████████████████████████████| 11.3 MB 8.7 MB/s 
  Attempting uninstall: pandas
    Found existing installation: pandas 1.1.5
    Uninstalling pandas-1.1.5:
      Successfully uninstalled pandas-1.1.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas~=1.1.0; python_version >= "3.0", but you have pandas 1.3.5 which is incompatible.


     |████████████████████████████████| 311 kB 9.8 MB/s 
     |████████████████████████████████| 1.1 MB 70.0 MB/s 
     |████████████████████████████████| 133 kB 75.0 MB/s 
     |████████████████████████████████| 243 kB 75.3 MB/s 
     |████████████████████████████████| 67 kB 6.6 MB/s 
     |████████████████████████████████| 144 kB 94.0 MB/s 
     |████████████████████████████████| 94 kB 2.8 MB/s 
     |████████████████████████████████| 271 kB 77.0 MB/s 
     |████████████████████████████████| 3.4 MB 7.9 MB/s 
     |████████████████████████████████| 895 kB 53.7 MB/s 
     |████████████████████████████████| 3.3 MB 51.1 MB/s 
     |████████████████████████████████| 596 kB 68.3 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from datasets import Dataset
import pickle

print("Loading data..", end='')
with open(f"/content/drive/MyDrive/GPT2/INFOMPR/pickles/cpt_2_data.p", "rb") as file:
          data = pickle.load(file)
print("Done")

dataset = Dataset.from_pandas(data)
dataset = dataset.select(range(int(36_000*1.25*1.25)))
print(dataset)

dataset = dataset.train_test_split(train_size=0.8, seed=42)
dataset2 = dataset['train'].train_test_split(train_size=0.8, seed=42)

dataset['train'] = dataset2.pop('train')
dataset['validation'] = dataset2.pop('test')

dataset = dataset.remove_columns(["reviewText",'startLabels','__index_level_0__'])

Loading data..Done
Dataset({
    features: ['cpt_input', 'reviewText', 'startLabels', '__index_level_0__'],
    num_rows: 56250
})


In [ ]:
print(dataset)
i = 0
for point in dataset['train']['cpt_input']:
  print(point)
  if i > 10:
    break
  i += 1

DatasetDict({
    train: Dataset({
        features: ['cpt_input'],
        num_rows: 3200
    })
    test: Dataset({
        features: ['cpt_input'],
        num_rows: 1000
    })
    validation: Dataset({
        features: ['cpt_input'],
        num_rows: 800
    })
})
<ReviewPrompt> Live in Tennessee VHS <review>What a crowd of fans ,what a show ..George interacted with the fans all through the performance. The band was at it's best , I have seen George in person several times and this was no different .Rest in peace George Jones as your music plays on .I own a piece of Country Music history .
<ReviewPrompt> Contraband Ed <review>now i usually don't gloat about how great or wonderful albums are, since most aren't that glamerous. velvet revolver is one of those rare groups that stitch together members of old bands trying to make comebacks and they sound better than they did before.

"contraband" is easily, without a shadow of a doubt, one of the best rock albums to be released in at 

In [ ]:
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel
from transformers import Trainer, TrainingArguments
from transformers import DataCollatorForLanguageModeling

### TRAINING
model_name = "gpt2"

torch.manual_seed(42)

tokenizer = GPT2Tokenizer.from_pretrained(model_name, bos_token='<ReviewPrompt>',
                                            eos_token='<endreview>', pad_token='<pad>', padding='max_length')
model = GPT2LMHeadModel.from_pretrained(model_name)
model.resize_token_embeddings(len(tokenizer))


def tokenize_function(example):
    return tokenizer(example["cpt_input"], truncation=True)

dataset = dataset.map(tokenize_function, batched=True)

training_args = TrainingArguments(output_dir='results', num_train_epochs=2, logging_steps=10,
                                    load_best_model_at_end=False,
                                    save_strategy='epoch',
                                    #per_device_train_batch_size=2, per_device_eval_batch_size=2,
                                    weight_decay=0.01, logging_dir='logs',
                                    evaluation_strategy="epoch"
                                  )


data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer, mlm=False,
    )
print("Training")
trainer = Trainer(model=model, args=training_args, train_dataset=dataset['train'],
        eval_dataset=dataset['validation'], 
        tokenizer=tokenizer,
        data_collator = data_collator)
trainer.train()


### TESTING
_ = model.eval()
print(dataset)


Downloading:   0%|          | 0.00/0.99M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/665 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


...

  0%|          | 0/9 [00:00<?, ?ba/s]

Training


The following columns in the training set  don't have a corresponding argument in `GPT2LMHeadModel.forward` and have been ignored: cpt_input.
***** Running training *****
  Num examples = 36000
  Num Epochs = 2
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 9000


Epoch,Training Loss,Validation Loss


RuntimeError: ignored

In [ ]:
trainer.save_model()

Saving model checkpoint to results
Configuration saved in results/config.json
Model weights saved in results/pytorch_model.bin
tokenizer config file saved in results/tokenizer_config.json
Special tokens file saved in results/special_tokens_map.json
added tokens file saved in results/added_tokens.json


In [ ]:
from transformers import pipeline
pipe = pipeline("text-generation", model="./results/checkpoint-36000", tokenizer=tokenizer)
example = dataset['test'].select(range(1))['cpt_input'][0][:66] 

string = "<reviewPrompt> The Highlights - The Weekend <review>"
new = pipe(string, max_length=400, min_length=300)
print(new)

loading configuration file ./results/checkpoint-36000/config.json
Model config GPT2Config {
  "_name_or_path": "./results/checkpoint-36000",
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 12,
  "n_positions": 1024,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "task_specific_params": {
    "text-generation": {
      "do_sample": true,
      "max_length": 50
    }
  },
  "torch_dtype": "float32",
  "transformers_version": "4.15.0",
  "use_cache": true,
  "vo

[{'generated_text': '<reviewPrompt> The Highlights - The Weekend <review>A classic! I have owned this disc and have been listening to it over and over.  For me, the standout songs on this CD are: "It\'s Going To Be a Dream", "He\'s Gonna Be a Hunter" -- The entire CD is great, with lots of upbeat songs ("You Know What You can do"), and plenty of rousing, and fun, ballads.  This just doesn\'t quite work for me.  I\'m always surprised that so many of my favorite artists have never really been out in the public arena, because I already love the best artists from my generation.\n"You See What I Saw (It Can Only Be) Is Hard to Find (Live)" (a great tune) is just another fun song.  It also has a great guitar solo by The Edge during the track. "Do You Care" has an almost bluesy feel to it, with the Edge\'s bass in the middle.\n"When Did You Feel (It Can Only Be So)" has some great, bluesy and fun music, and then the song ends with some wonderful acoustic guitar.\nThe rest of the disc is mostl

In [ ]:
print(len("<reviewPrompt> I Love You The Neighbourhood <review>I first noticed them in a small nightclub in Los Angeles, when I was young. They weren't as appealing as other bands, but they took it's guesswork out on"))
print(len("<reviewPrompt> Daddy Issues The Neighbourhood <review>Sarajevo's brother, who plays bassist at the rock quartet Skillet, plays a guitar that's quite different to his brother's. He makes new material"))
print(len("<reviewPrompt> KOD J. Cole <review>great songs and you will enjoy this concert!  they sound great!!\n\njust wish I could buy it for them but they could do better to get it at a good price."))

205
198
186


In [ ]:
raw_inputs = [
    "<reviewPrompt> The Highlights - The Weekend <review>"
]

inputs = tokenizer(raw_inputs, padding=True, truncation=True, return_tensors="pt").cuda()
outputs = model(**inputs)


RuntimeError: ignored

In [ ]:
input_sequence = "<reviewPrompt> The Highlights - The Weekend <review>"
input_ids = tokenizer.encode(input_sequence, return_tensors='pt').cuda()
output = model.generate(input_ids, max_length=1000, min_length=500)
print(tokenizer.decode(output[0]))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


<reviewPrompt> The Highlights - The Weekend <review>I love this album.  I love the songs.  I love the production.  I love the lyrics.  I love the music.  I love the production.  I love the lyrics.  I love the music.  I love the production.  I love the lyrics.  I love the music.  I love the production.  I love the lyrics.  I love the music.  I love the production.  I love the lyrics.  I love the music.  I love the production.  I love the lyrics.  I love the music.  I love the production.  I love the lyrics.  I love the music.  I love the production.  I love the lyrics.  I love the music.  I love the production.  I love the lyrics.  I love the music.  I love the production.  I love the lyrics.  I love the music.  I love the production.  I love the lyrics.  I love the music.  I love the production.  I love the lyrics.  I love the music.  I love the production.  I love the lyrics.  I love the music.  I love the production.  I love the lyrics.  I love the music.  I love the production.  I l